In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Wed Oct 24 14:31:18 2018

@author: jimmy
"""

#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Fri Oct 19 14:51:07 2018

@author: jeffchen
"""

import numpy as np
import torch
import torch.utils.data
import json

###split a sentence 
def sent2words(sent):
    res_para = [j.split(' ') for j in (chr(ord(sent[0])+32) + sent[1:-1]).split(',')]
    res_tok = res_para.pop(0)
    while len(res_para) != 0:
        res_tok.extend([','] + res_para.pop(0)[1:])
    #res_tok += ['.']
    for j in range(len(res_tok)):
        if res_tok[j][-2:] == "'s" and res_tok[j] != "'s":
            res_tok[j] = res_tok[j][:-2]
            res_tok.insert(j+1,"'s")
    return res_tok

def generate_word_dict(directory = "./MLDS_hw2_1_data/"):
    print("Loading Data")
    ###Load data
    data_direc = directory
    # Training part
    train_labels, train_feat_dict = [], {}
    with open(data_direc + "training_label.json", "r") as f_train:
        train_labels = json.loads(f_train.read())
        for i in train_labels:
            train_feat_dict[ i['id'] ] = np.load(data_direc + 'training_data/feat/' + i['id'] + '.npy')
    
    print(len(train_labels))
    ###Build word dictionary and pop some words
    word_dict = {}
    
    for i in range(len(train_labels)):
        caption_list = train_labels[i]['caption']
        for j in range(len(caption_list)):
            tokens = sent2words(caption_list[j])
            for k in tokens:
                if k not in word_dict.keys():
                    word_dict[k] = 1
                else:
                    word_dict[k] += 1
            tokens.append('<EOS>')
            train_labels[i]['caption'][j] = tokens
    to_pop = []
    for i in word_dict.keys():
        if word_dict[i] < 3:
            to_pop.append(i)
    for i in to_pop:
        word_dict.pop(i)
    print("Finish count words")
    int_to_dict = {}
    for num, key in enumerate(word_dict.keys()):
        int_to_dict[num] = key
    n_words = len(int_to_dict)
    int_to_dict[n_words] = '<UNK>'
    int_to_dict[n_words+1] = '<BOS>'
    int_to_dict[n_words+2] = '<EOS>'
    return int_to_dict

def generate_test_dataloader(directory = "./MLDS_hw2_1_data/", batch_size = 16):
    print("Loading Data")
    ###Load data
    data_direc = directory
      
    # Testing part
    test_labels, test_feat_dict = [], {}
    with open(data_direc + "testing_label.json", "r") as f_test:
        test_labels = json.loads(f_test.read())
        for i in test_labels:
            test_feat_dict[ i['id'] ] = np.load(data_direc + 'testing_data/feat/' + i['id'] + '.npy')
    print(len(test_labels))
    ###Build word dictionary and pop some words
    
    test_x = []
    video_id_list = []
    for data_num in range(len(test_labels)):
        print("Process data ",data_num," to list")

        video_id = test_labels[data_num]['id']
        video_id_list.append(video_id)
        test_x.append(test_feat_dict[video_id]) 
 
    test_x = np.array(test_x)
    
    print("Convert to Tensor")
    tensor_test_x = torch.Tensor(test_x)
    #dataset = torch.utils.data.TensorDataset((tensor_test_x))
    #dataloader = torch.utils.data.DataLoader(dataset = dataset, batch_size = batch_size, shuffle=False)
    return tensor_test_x, video_id_list

def prediction(model_dir, input_x, id_list, int_to_word):
    file = open("TEST.txt",'w')
    model = torch.load(model_dir).cuda()
    for i in range(len(input_x)):
        input_data = input_x[i].reshape(80, 1, 4096).cuda()
        sentence = ""
        ans = model.test(input_data, 44)
        #print(ans[0])
        for j in range(len(ans)):
            one_word = int_to_word[torch.argmax(ans[j]).item()]
            if(one_word == '<EOS>'):
                print("Have EOS")
                break
            sentence += " "+one_word
        file.write(id_list[i]+','+sentence+'\n')
    file.close()

In [2]:
int_to_word = generate_word_dict()

Loading Data
1450
Finish count words


In [3]:
int_to_word[2889]

'<EOS>'

In [4]:
test_data, id_list = generate_test_dataloader()

Loading Data
100
Process data  0  to list
Process data  1  to list
Process data  2  to list
Process data  3  to list
Process data  4  to list
Process data  5  to list
Process data  6  to list
Process data  7  to list
Process data  8  to list
Process data  9  to list
Process data  10  to list
Process data  11  to list
Process data  12  to list
Process data  13  to list
Process data  14  to list
Process data  15  to list
Process data  16  to list
Process data  17  to list
Process data  18  to list
Process data  19  to list
Process data  20  to list
Process data  21  to list
Process data  22  to list
Process data  23  to list
Process data  24  to list
Process data  25  to list
Process data  26  to list
Process data  27  to list
Process data  28  to list
Process data  29  to list
Process data  30  to list
Process data  31  to list
Process data  32  to list
Process data  33  to list
Process data  34  to list
Process data  35  to list
Process data  36  to list
Process data  37  to list
Proce

In [5]:
import S2VT

model = torch.load('./s2vt_1025_2_old.pkl')
model.batch_size = 1
model.encoder_h = torch.zeros((1, 1, 256),
                                      dtype = torch.float32).cuda()
model.encoder_c = torch.zeros((1, 1, 256),
                                      dtype = torch.float32).cuda()
model.decoder_h = torch.zeros((1, 1, 256),
                                      dtype = torch.float32).cuda()
model.decoder_c = torch.zeros((1, 1, 256),
                                      dtype = torch.float32).cuda()
torch.save(model, './s2vt_1025_2_old_MODEL.pkl')

prediction('./s2vt_1025_2_old_MODEL.pkl', test_data, id_list, int_to_word)

/opt/conda/lib/python3.6/site-packages/torch/serialization.py:425: SourceChangeWarning: source code of class 'S2VT.S2VT' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/opt/conda/lib/python3.6/site-packages/torch/serialization.py:425: SourceChangeWarning: source code of class 'S2VT.Attention' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)


Encoding
Decoding
Have EOS
Encoding
Decoding
Have EOS
Encoding
Decoding
Have EOS
Encoding
Decoding
Have EOS
Encoding
Decoding
Have EOS
Encoding
Decoding
Have EOS
Encoding
Decoding
Have EOS
Encoding
Decoding
Have EOS
Encoding
Decoding
Have EOS
Encoding
Decoding
Have EOS
Encoding
Decoding
Have EOS
Encoding
Decoding
Have EOS
Encoding
Decoding
Have EOS
Encoding
Decoding
Have EOS
Encoding
Decoding
Have EOS
Encoding
Decoding
Have EOS
Encoding
Decoding
Have EOS
Encoding
Decoding
Have EOS
Encoding
Decoding
Have EOS
Encoding
Decoding
Have EOS
Encoding
Decoding
Have EOS
Encoding
Decoding
Have EOS
Encoding
Decoding
Have EOS
Encoding
Decoding
Have EOS
Encoding
Decoding
Have EOS
Encoding
Decoding
Have EOS
Encoding
Decoding
Have EOS
Encoding
Decoding
Have EOS
Encoding
Decoding
Have EOS
Encoding
Decoding
Have EOS
Encoding
Decoding
Have EOS
Encoding
Decoding
Have EOS
Encoding
Decoding
Have EOS
Encoding
Decoding
Have EOS
Encoding
Decoding
Have EOS
Encoding
Decoding
Have EOS
Encoding
Decoding
Have EOS
E

In [10]:
test_data[2]


tensor([[0.2367, 1.1624, 0.0000,  ..., 0.0000, 0.0000, 0.4624],
        [0.0000, 1.2430, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 1.0580, 0.0000,  ..., 0.0000, 0.0000, 0.1048],
        ...,
        [1.6902, 2.8036, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [1.7786, 2.6784, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [1.9601, 2.7707, 0.0000,  ..., 0.0000, 0.0000, 0.0000]])